# Wykład 7 - Elementy Programowania Funkcyjnego

## 7.1 val vs fun

In [20]:
val <T> List<T>.head: List<T>
    get() = take(2)

In [21]:
fun <T> List<T>.head2(): List<T> = take(2)

In [27]:
listOf(1, 2, 3, 4).head

[1, 2]

In [23]:
listOf(1, 2, 3, 4).head2()

[1, 2]

- **`val`** - opisuje stan
- **`fun`** - opisuje zachowanie

In [10]:
class Student (var firstName:String, var lastName:String, var avg:Double){
    fun fullName() = "$firstName $lastName"
}

In [28]:
class Student2 (var firstName:String, var lastName:String, var avg:Double){
    val fullName = "$firstName $lastName"
}

val s = Student2("Rafał", "Lewandków", 3.3)
println(s.fullName)

Rafał Lewandków


In [29]:
s.firstName = "Paweł"
println(s.fullName)

Rafał Lewandków


In [31]:
s.firstName

Paweł

W klasie `Student2` właściwość jest obliczone dokładnie jeden raz - przy tworzeniu obiektu.

In [12]:
class Student3 (var firstName:String, var lastName:String, var avg:Double){
    val fullName
        get() = "$firstName $lastName"
}

In [33]:
val student = Student3("Rafał", "Lewandków", 3.3)
println(s.fullName)

Rafał Lewandków


In [34]:
student.firstName = "Paweł"
println(student.fullName)

Paweł Lewandków


In [35]:
class Student4 (var firstName:String, var lastName:String, var avg:Double){
    val fullName
        get() = "$firstName $lastName"
    
    fun exam(): Double {
        val grade = 3.5
        avg = grade
        return grade
    }
}

Wskazówki - Kiedy używać `val`:
- gdy nie rzucamy wyjątku
- tani obliczeniowo
- jeżeli stan obiektu się nie zmienił, zawsze zwraca tą samą wartość

In [36]:
class Student5 (var firstName:String, var lastName:String, var avg:Double){
    val fullName
        get() = "$firstName $lastName"
    
    fun exam(): Double {
        val grade = 3.5
        avg = grade
        return grade
    }
}

## 7.2 Pętle vs Rekurencja

Wykorzystanie funkcji aby pozbyć się mutowalności iteracyjnej

Pętle wykorzystują efekty uboczne

In [40]:
val a = listOf(1, 2, 3, 4)

fun suma(l: List<Int>): Int{ // czysta funkcja
    var sum = 0
    for (i in l){
        sum += i // efekt uboczny - modyfikacja stanu
    }
    return sum
}
println(suma(a))

10


## 7.3 Inline fun

In [1]:
class A {
    fun function1(i:Int, f:(Int) -> String): String {
        return f(i)
    }
    
    fun function2() {
        val a = 7
        val s = function1(8) {
            i -> "8 + a = " + (i+a) 
        }
    }
}

W wywołaniu `function1` przekazujemy funkcje w postaci lambdy, jest to obiekt który musi zostać utworzony w czasie wykonania. Dodatkowo właściwość `a` musi zostać przekazana do tego obiektu.

In [2]:
class A {
    inline fun function1(i:Int, f:(Int) -> String): String
    {
        return f(i)
    }
    fun function2() {
        val a = 7
        val s = function1(8) {
            i -> "8 + a = " + (i+a) }
    }
}

Gdy funkcja `inline` zostaje wykorzystana, cały kod funkcji zostaje skopiowany w miejsce wywołania

Prawie wszystkie funkcje wyższego rzędu w `stdlib` są `inline`

In [3]:
inline fun repeat(times: Int, action: (Int) -> Unit) {
    for (index in 0 until times) {
        action(index)
    }
}

In [4]:
repeat(10) {
    print(it)
}

0123456789

Powyższe wywołanie zostanie zastąpione przez

In [5]:
for (index in 0 until 10) {
    print(index)
}

0123456789

Wykonanie funkcji przyjmujące funkcję jako parametr są szybsze gdy są `inline`.

## 7.4 Funkcja przyjmująca jako parametr funkcję - Java

In [3]:
interface Callable {
    public void call(int param);
}
    
class Test implements Callable {
    public void call(int param) {
        System.out.println("Called with " + param );
    }
}
    
public class HelloWorld{
    public static void invoke(Callable callable, int param){
        callable.call(param);
    }
    
     public static void main(){
        Callable cmd = new Test();
        invoke(cmd, 100);
         
         // klasa anonimowa
        // invoke(new Callable(){
        //     public void call(int param){
        //         System.out.println("Called with " + param );
        //     }
        // }, 100);
         
        // invoke((param -> System.out.println("Called with " + param )), 100);

    }
}
HelloWorld.main()

Called with 100
Called with 100


In [6]:
import java.util.function.Function;

public class HigherOrderFunc {

    public static void main() {

        //Function <Integer, Integer> inc = e -> e + 1;
        doSum(5, e -> e + 1);

    }

    public static void doSum(int value, Function <Integer, Integer> func) {
        System.out.println(func.apply(value));
    }
}

HigherOrderFunc.main()

6


## 7.5 Operator * i varargs

In [2]:
fun format(format: String, vararg params: String){}

Jeżeli `varargs` jest podany jako ostatni parametr, jego odpowiednikiem w Javie jest

In [ ]:
void format(@NotNull String format, @NotNull String... params)

In [5]:
fun format2(format: String, vararg params: String, encoding: String){}

Jeżeli `varargs` nie jest ostatnim paramtrem, odpowiednikiem w Javie jest

In [ ]:
void format2(String format, String[] params, String encoding)

In [7]:
fun <T> printAll(vararg ts: T) {
    ts.forEach { println(it) }
}

In [11]:
val numbers = arrayOf(1, 2, 3, 4)
printAll(*numbers)

1
2
3
4


In [13]:
val params = arrayOf("param1", "param2")
val opts = arrayOf("opts1", "opts2", "opts3")

val allParams = arrayOf(*params, "extra", *opts)
printAll(*allParams)

param1
param2
extra
opts1
opts2
opts3


## 7.6 Scope functions

### `let`

In [15]:
var number: Int? = null

if (number != null){
    var num = number + 2
}


Line_14.jupyter-kts (4:9 - 12) Variable 'num' is never used
Line_14.jupyter-kts (4:15 - 21) Smart cast to 'Int' is impossible, because 'number' is a mutable property that could have been changed by this time

In [21]:
var number: Int? = null

// niebezpieczne ze względu na wielowątkowość
if (number != null){
    var num = number!! + 2
    println(num)
}

In [24]:
var number: Int? = null

// bezpieczne ze względu na wielowątkowość
// let jest wykonywany tylko gdy number? nie jest null
number?.let{ // it zachowuje stan number
    var num = it + 2 // it == number
    println(num)
}

null

In [31]:
var number: Int? = 0
// zwracana wartość ostatniej linii
val v = number?.let{
    var num = it + 2 
    num
}

println(v)

2


In [2]:
var numberx: Int? = null
// zwracana wartość ostatniej linii
val va = numberx?.let{it + 2}
println(va)

null


In [3]:
var numberx: Int? = null
// zwracana wartość ostatniej linii
val va = numberx?.let{it + 2}?:3
println(va)

3


In [20]:
var a = 1
var b= 2

a = a
    .let { it + 2 }
    .let { 
    val i = it + b
    i
}
println(a)

5


### `also`

`also` nie zwraca ostatniej linii jak `let`. Zwraca obiekt na którym został wywołany.

In [11]:
val numbers = mutableListOf("one", "two", "three")
numbers
    .also { println("The list elements before adding new one: $it") }
    .add("four")
    .also { println("The list elements after adding new one: $it") }

The list elements before adding new one: [one, two, three]
The list elements after adding new one: true


true

### `apply`

In [17]:
data class Person(var name: String, var age: Int=0, var city: String="")
val adam = Person("Adam").apply {
    age = 32
    city = "London"        
}
println(adam)

Person(name=Adam, age=32, city=London)


### `run`

In [ ]:
val service = MultiportService("https://example.kotlinlang.org", 80)

val result = service.run {
    port = 8080
    query(prepareRequest() + " to port $port")
}

// the same with let()
val letResult = service.let {
    it.port = 8080
    it.query(it.prepareRequest() + " to port ${it.port}")
}

In [25]:
var imie = "Rafał"
println(imie)
    
imie = run {
    val tutorial = "Paweł"
    tutorial
}
println(imie)
    
tutorial = run {
    val imie = "Paweł"
    tutorial
}

imie = run {
    "Radek"
    "Gadek"
}
println(imie)

Rafał
Paweł
Gadek


In [31]:
var p : String? = null
p?.let { 
    println("p is $p") } ?: run { 
    print("default value: ")
        p = "Kotlin"
}

println(p)

default value: Kotlin


In [33]:
data class Person(var name: String, var lastName : String)
var person = Person("Rafał", "Lewandków")

var r = person.let { it.lastName = "Lewandkowski" }
var rl = person.also { it.lastName = "Lewandkowski" }
    
println(r)
println(rl)
println(person)

kotlin.Unit
Person(name=Rafał, lastName=Lewandkowski)
Person(name=Rafał, lastName=Lewandkowski)


In [34]:
data class Person(var n: String, var l : String)
var person = Person("Rafał", "Lewandków")

person.apply { l = "Lewandkowski" }
println(person)

person.also { it.l = "Lewandkowski" }
println(person)

Person(n=Rafał, l=Lewandkowski)
Person(n=Rafał, l=Lewandkowski)


### `with`

In [36]:
data class Person(var name: String, var lastName : String)
var person = Person("Rafał", "Lewandków")

with(person)
    {
        name = "No Name"
        lastName = "No Name"
    }

println(person)

Person(name=No Name, lastName=No Name)


### `with` vs `apply`

- `apply` działa na instancji obiektu, `with` dostaje obiekt jako argument
- `with` zwraca ostatnią linię